### Canned laughter identification model using spectrograms of sound

In this notebook, we will train a model based on friends laughter/non-laughter spectrograms

In [ ]:
import sys
sys.path.append('../utils/')

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
# local imports
import utils
%aimport soundutils
%aimport episode
import color
import stats
%aimport modelbuilder
# stdlib and package imports
import pydub
import numpy as np
import pandas as pd
from pathlib import Path 
from matplotlib import pyplot as plt
import seaborn
from collections import Counter
from scipy import signal
# keras and ML imports
from keras.models import Sequential, Model, model_from_yaml
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize as sknormalize
from sklearn.metrics import confusion_matrix
from imblearn import under_sampling
from scipy.io import wavfile

### Spectrograms
In this notebook we'll extract spectrograms of sound data and try to train a model using these instead of the raw audio

#### Step 1. Explore `scipy.signal.spectrogram` and write useful wrapper methods in our library

In [ ]:
ep = Path('../wav/') / 'friends-s03-e09.wav'

In [ ]:
sr, wav = wavfile.read(str(ep))

In [ ]:
win = 9 # 9th chunk
wid = 1 # 1 second width
f, t, Sxx = signal.spectrogram(wav[win*wid*sr:(win+1)*wid*sr], sr)

Here's what the default output produced by scipy would look like. However, we don't want so much data,
so we'll use a wrapper to downsample by smoothing it over sliding windows

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.pcolormesh(t, f[:100], np.log10(1+Sxx[:100]))
plt.ylabel('log Frequency [Hz]')
plt.xlabel('Time [sec]')
fig.show()

#### Test a wrapper method
`soundutils.get_data_spectro`:
- `wavdata`: raw wav audio data
- `sr`: sampling rate of wav
- `windowlen`: length of sliding window to smooth over (milliseconds)
- `fn`: a function that will take an (n, windowlen) np array and return a (n,) np array which is the smoothed version

In [ ]:
f, t, samples = soundutils.get_data_spectro(wavdata=wav[win*wid*sr:(win+1)*wid*sr], sr=sr, windowlen=16,
                                            fn=lambda x: np.mean(x, axis=1))

In [ ]:
samples.shape

We should see a smoother plot than before 

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.pcolormesh(t, f, samples.T)
plt.ylabel('log Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

### Now with a method in place, we try it out with a whole episode

In [ ]:
# these are the episodes we have annotation data for
episodes = ['friends-s02-e{:0>2d}'.format(i) for i in range(1, 5)] + ['friends-s03-e09']

In [ ]:
X_raw, Y_raw, refs = episode.get_data(which_episodes=episodes, backend='spectro', task='laughter', windowlen=100)

In [ ]:
Counter(refs)

In [ ]:
# for i in range(145, 155):
fig = plt.figure(figsize=(15,5))
plt.pcolormesh(np.array(X_raw[0:3000]).T.reshape(129,-1)[:100])
plt.show()
fig = plt.figure(figsize=(15,5))
plt.pcolormesh(np.array(X_raw[11401:14401]).T.reshape(129,-1)[:100])
plt.show()

### Train a model on it

#### Now we'll use the extracted data to generate balanced training and testing data sets
First, resample data to have equal number of 'laugh' and 'no-laugh' examples

In [ ]:
szn_num = [int(x[9:11]) for x, _, _ in refs]
train_flag = np.array([x in [2] for x in szn_num])
print(Counter(train_flag))

X_raw_train = X_raw[train_flag,]
X_raw_valid = X_raw[~train_flag,]
Y_raw_train = Y_raw[train_flag,]
Y_raw_valid = Y_raw[~train_flag,]

rus = under_sampling.RandomUnderSampler(sampling_strategy='not minority')
X_train, Y_train = rus.fit_resample(X_raw_train, Y_raw_train)
X_valid, Y_valid = rus.fit_resample(X_raw_valid, Y_raw_valid)

print(Counter(Y_train.flatten()))
print(Counter(Y_valid.flatten()))

In [ ]:
X_train = X_train.reshape(*X_train.shape, 1)
X_valid = X_valid.reshape(*X_valid.shape, 1)

X_train.shape

#### Now we'll attempt to model the balanced data using a Convolutional model

In [ ]:
checkpoint = ModelCheckpoint(filepath='task:{task}-spectro-ckpt.hdf5'.format(task='train-on-s02'),
                             save_best_only=True)
model = modelbuilder.build_conv_model(optimizer='rmsprop', drop1=.3)

In [ ]:
# train model
H = model.fit(X_train, Y_train.reshape(-1), epochs=10, 
              validation_data=[X_valid, Y_valid.reshape(-1)], callbacks=[checkpoint])

In [ ]:
stats.plot_history(H)

## Fitting a logistic regression model

How well does a logistic regression model do in making predictions
of laughter with the spectrogram data? I'll fit a model with an l1
penalty (lasso regression) and set the penalty parameter to a relatively
small value (C=0.01) corresponding with a high amount of penalization.

In [ ]:
import sklearn

model = sklearn.linear_model.LogisticRegression(penalty="l1", C=0.01)
model = model.fit(X_train[:, :, 0], Y_train)

The model does very similarly to the neural network model. It has about
a 76% classification rate. You can get a bit higher with a larger value
of C (around 78%), but I like the small model for illustration purposes.

In [ ]:
model.score(X_valid[:, :, 0], Y_valid)

In [ ]:
yhat_logit = model.predict_proba(X_valid[:, :, 0])[:,1]
fpr, tpr, thr = sklearn.metrics.roc_curve(Y_valid, yhat_logit)
plt.plot(fpr, tpr, 'b')

And here are the coefficents. Notice that only the first 30ish values come
into the model; this matches what we saw when looking at the data manually
(i.e., the signal comes from the smaller frequencies).

In [ ]:
model.coef_

Also notice that the model is doing well balancing the predictions:

In [ ]:
yhat_logit = model.predict_proba(X_valid[:, :, 0])[:,1]
_ = plt.hist(yhat_logit, bins=100)

How correlated are the sets of predictions? Relatively high, but
not overly so (r=0.84 on my run of the dataset).

In [ ]:
yhat_keras = H.model.predict(X_valid[:, :, :])[:,0]
yhat_logit = model.predict_proba(X_valid[:, :, 0])[:,1]

In [ ]:
np.corrcoef(yhat_keras, yhat_logit)

In [ ]:
sklearn.metrics.confusion_matrix(yhat_keras > 0.5, yhat_logit > 0.5)

### This code should not be trusted

Saving the code below, but doesn't mean much unless we can sort the dataset.

In [ ]:
np.corrcoef(yhat_keras[1:], yhat_keras[:-1])

In [ ]:
np.corrcoef(yhat_logit[1:], yhat_logit[:-1])

In [ ]:
# Smoothed NN model
for size in [1, 5, 10, 50, 100, 500, 1000]:
    weights = [1 / size] * size
    yhat_keras_smooth = np.convolve(yhat_keras, np.array(weights)[::-1],'same')
    acc = np.mean(np.int32(yhat_keras_smooth > 0.5) == np.int32(Y_valid[:,0]))
    print("size = {0: 5d}    acc = {1:01.04f}".format(size, acc))

In [ ]:
# Smoothed Logistic Model
for size in [1, 5, 10, 50, 100, 500, 1000, 6500]:
    weights = [1 / size] * size
    yhat_logit_smooth = np.convolve(yhat_logit, np.array(weights)[::-1], 'same')
    acc = np.mean(np.int32(yhat_logit_smooth > 0.5) == np.int32(Y_valid[:,0]))
    
    print("size = {0: 5d}    acc = {1:01.04f}".format(size, acc))